In [2]:
from PIL import Image
import os
import torch
import hashlib
import tarfile
import requests
import numpy as np
from tqdm import tqdm
from PIL import Image
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision.transforms import v2
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from dataset import LFWDataset
from io import BytesIO
import requests
import gradio as gr

In [3]:
my_model = torch.jit.load("var2_scripted_model.pt")

In [4]:
def segment_image(input_image):
    image_array = np.array(input_image)
    image_tensor = torch.unsqueeze(torch.transpose(torch.tensor(input_image), 0, 2), 0).float()
    
    with torch.no_grad():
        output = my_model(image_tensor)

    predicted_mask = torch.argmax(output, dim=1).squeeze().numpy()
    predicted_mask_rotated = np.rot90(predicted_mask, k=-1)
    predicted_mask_final = np.fliplr(predicted_mask_rotated)
    
    predicted_mask_final = (predicted_mask_final * 255).astype(np.uint8)
    return predicted_mask_final

# Gradio Interface
ui = gr.Interface(
    fn=segment_image, 
    inputs=gr.Image(),
    outputs=gr.Image(),
    examples=['celeb_inputs/celeb1.jpg', 
              'celeb_inputs/celeb2.jpg', 
              'celeb_inputs/celeb3.jpg',
              'celeb_inputs/celeb4.jpg',
              'celeb_inputs/celeb5.jpg']
)

In [5]:
ui.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
